# Reconhecimento do Estado do Jogo Elo Maluco Usando HOG e Processamento de Imagens

## Descrição
Este projeto implementa um sistema para:

- Detectar padrões em imagens utilizando características HOG (Histogram of Oriented Gradients) e similaridade estrutural (SSIM).
- Identificar a cor predominante nas imagens com base em faixas HSV.
- Processar imagens do "Elo Maluco", dividindo-as em regiões de interesse (faces), identificando padrões e cores em cada face, e gerando uma representação do estado em formato XML.

[Assista ao vídeo de apresentação aqui](https://youtu.be/MP8NgF96cWI)




<div style="padding: 0px 50px; ">
    <img src="./DiagramaDetalhado.jpeg" alt="" width="700px" height="500px">
</div>



## Dependências
Para executar este código, as seguintes bibliotecas são necessárias:

- **Python 3.8+**
- **OpenCV**: Para manipulação e processamento de imagens.
  ```bash
  pip install opencv-python
  ```
- **NumPy**: Para operações matemáticas e manipulação de arrays.
  ```bash
  pip install numpy
  ```
- **scikit-image**: Para cálculo de SSIM e extração de HOG.
  ```bash
  pip install scikit-image
  ```
- **xml.etree.ElementTree**: Biblioteca padrão para manipulação de XML (inclusa no Python).
- **xml.dom.minidom**: Para formatação de XML (inclusa no Python).

## Estrutura do Projeto
- **`decodeEloMaluco.ipynb`**: Arquivo principal contendo a implementação das classes e funções, além da descrição do projeto.
- **Diretório `templates`**: Contém as imagens de referência (`.png`) para os padrões "superior", "intermediaria", "inferior" e "vazia".
- **Diretório `data`**: Contém as imagens a serem processadas.

## Execução
1. Certifique-se de que as dependências estão instaladas.
2. Organize os arquivos conforme a estrutura descrita.
3. Execute as células de código **sequencialmente** em **`decodeEloMaluco.ipynb`** em um ambiente Python.
4. O script irá processar as imagens na pasta `data` e gerar um arquivo XML chamado `estado_atual.xml` com os resultados.

## Código

### Importação das Bibliotecas
Nesta etapa, importamos as bibliotecas necessárias para processamento de imagens, cálculo de HOG e SSIM, manipulação de XML e leitura de arquivos.

In [11]:
import cv2
import numpy as np
from pathlib import Path
from skimage.metrics import structural_similarity as ssim
from skimage.feature import hog
import xml.etree.ElementTree as ET
from xml.dom import minidom


### Classe PatternMatcher

A classe `PatternMatcher` é responsável por identificar padrões nas imagens do Elo Maluco. Ela utiliza características HOG (Histogram of Oriented Gradients) e medidas de similaridade estrutural (SSIM) para comparar as faces extraídas das imagens com templates predefinidos.

#### Funcionalidades principais:
1. **Carregamento de templates:** Os templates são imagens de referência que representam padrões como "superior", "intermediaria", "inferior" e "vazia".
2. **Extração de características HOG:** Cada template é processado para gerar descritores HOG, que são usados na comparação com as faces extraídas.
3. **Pré-processamento:** As imagens são convertidas para tons de cinza, redimensionadas e passam por técnicas de realce de contraste e redução de ruído.
4. **Identificação de padrões:** Ao processar uma imagem, a classe calcula a similaridade entre as faces extraídas e os templates, determinando o padrão mais similar.

#### Principais métodos:
- `__init__`: Inicializa a classe e carrega os templates.
- `load_templates`: Carrega os templates a partir do diretório especificado.
- `preprocess_face`: Pré-processa as imagens para uniformizar tamanho, contraste e qualidade.
- `extract_hog_features`: Extrai características HOG de uma imagem.
- `match_pattern`: Encontra o padrão mais similar em uma face processada, usando HOG e SSIM.


In [12]:
class PatternMatcher:
    def __init__(self, templates_dir='templates'):
        """
        Inicializa o detector de padrões com HOG e threshold para faces vazias
        """
        self.templates = {
            'superior': None,
            'intermediaria': None,
            'inferior': None,
            'vazia': None
        }
        self.template_size = (150, 150)
        self.hog_features = {}
        self.empty_threshold = 5  # Threshold para considerar face vazia
        self.load_templates(templates_dir)
    
    def load_templates(self, templates_dir):
        """
        Carrega e extrai características HOG dos templates
        """
        path = Path(templates_dir)
        if not path.exists():
            print(f"Diretório de templates '{templates_dir}' não encontrado")
            return
            
        for pattern_name in self.templates.keys():
            template_path = path / f"{pattern_name}.png"
            if template_path.exists():
                template = cv2.imread(str(template_path))
                if template is not None:
                    processed_template = self.preprocess_face(template)
                    self.templates[pattern_name] = processed_template
                    
                    hog_features = self.extract_hog_features(processed_template)
                    self.hog_features[pattern_name] = hog_features
                    
                    print(f"Template '{pattern_name}' carregado com sucesso")
            else:
                print(f"Template '{pattern_name}' não encontrado em {template_path}")

    def extract_hog_features(self, image):
        """
        Extrai características HOG da imagem
        """
        fd = hog(image, 
                orientations=8,
                pixels_per_cell=(16, 16),
                cells_per_block=(2, 2), 
                visualize=False,
                feature_vector=True)
        return fd
    
    def preprocess_face(self, face_img):
        """
        Pré-processa uma face para análise
        """
        # Converter para escala de cinza
        gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
        
        # Redimensionar para tamanho padrão
        resized = cv2.resize(gray, self.template_size)
        
        # Melhorar contraste
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        enhanced = clahe.apply(resized)
        
        # Redução de ruído
        denoised = cv2.fastNlMeansDenoising(enhanced)
        
        # Binarização adaptativa
        binary = cv2.adaptiveThreshold(
            denoised,
            255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY,
            21,
            5
        )
        
        # Operações morfológicas
        kernel = np.ones((3,3), np.uint8)
        cleaned = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
        
        return cleaned
    
    def match_pattern(self, face_img, debug=False):
        """
        Encontra o padrão mais similar, considerando threshold para face vazia
        """
        processed_face = self.preprocess_face(face_img)
        face_hog = self.extract_hog_features(processed_face)
        
        similarities = {}
        
        # Calcular similaridades para todos os padrões
        for pattern_name, template in self.templates.items():
            if template is None:
                continue
            
            # Calcular similaridade SSIM
            ssim_score = ssim(processed_face, template)
            
            # Calcular similaridade HOG
            hog_correlation = np.correlate(face_hog, self.hog_features[pattern_name])[0]
            hog_score = (hog_correlation + 1) / 2  # Normalizar para [0,1]
            
            # Combinar scores
            combined_score = 0.6 * hog_score + 0.4 * ssim_score
            similarities[pattern_name] = combined_score
        
        if debug:
            print("\nSimilaridades encontradas:")
            for pattern, sim in similarities.items():
                print(f"{pattern}: {sim:.3f}")
        
        # Verificar se todos os scores são baixos (indicativo de face vazia)
        all_scores_low = all(score < self.empty_threshold 
                           for pattern, score in similarities.items() 
                           if pattern != 'vazia')
        
        if all_scores_low:
            return 'vazia', similarities.get('vazia', 0.0)
        
        # Caso contrário, retornar o padrão com maior similaridade
        best_match = max(similarities.items(), key=lambda x: x[1])
        return best_match[0], best_match[1]

### Classe EloColorDetector

A classe `EloColorDetector` é responsável por detectar a cor predominante nas faces extraídas das imagens do Elo Maluco. A identificação é feita com base em faixas de cores no espaço HSV (Hue, Saturation, Value).

#### Funcionalidades principais:
1. **Definição de faixas de cores:** A classe define intervalos HSV para as cores "amarelo", "verde", "vermelho" (em dois intervalos), e "branco". Esses intervalos são usados para criar máscaras que identificam pixels com a cor correspondente.
2. **Pré-processamento:** As imagens passam por redimensionamento e suavização para melhorar a precisão da detecção.
3. **Cálculo da porcentagem de pixels:** A classe calcula a porcentagem de pixels de cada cor dentro de uma face e determina qual cor é predominante.
4. **Filtro de área mínima:** Apenas cores que ocupam uma área mínima significativa são consideradas válidas.

In [13]:
class EloColorDetector:
    def __init__(self):
        """
        Inicializa o detector de cores com ranges HSV expandidos
        """
        # Definir ranges HSV para cada cor
        self.color_ranges = {
            'amarelo': {
                'lower': np.array([20, 100, 100]),
                'upper': np.array([35, 255, 255])
            },
            'verde': {
                'lower': np.array([35, 100, 100]),
                'upper': np.array([85, 255, 255])
            },
            'vermelho1': {  # Primeiro range do vermelho (0-10)
                'lower': np.array([0, 100, 100]),
                'upper': np.array([10, 255, 255])
            },
            'vermelho2': {  # Segundo range do vermelho (160-180)
                'lower': np.array([160, 100, 100]),
                'upper': np.array([180, 255, 255])
            },
            'branco': {
                'lower': np.array([0, 0, 200]),  # Baixa saturação, alto valor
                'upper': np.array([180, 30, 255])
            }
        }
        
        # Tamanho mínimo da área colorida para ser considerada (em % da face)
        self.min_color_area = 0.15
    
    def preprocess_for_color(self, face_img):
        """
        Pré-processa a imagem para detecção de cor
        """
        # Redimensionar para um tamanho padrão
        resized = cv2.resize(face_img, (150, 150))
        
        # Aplicar blur suave para reduzir ruído
        blurred = cv2.GaussianBlur(resized, (5, 5), 0)
        
        # Converter para HSV
        hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
        
        return hsv, blurred  # Retorna também a imagem borrada para detecção de branco
    
    def detect_color(self, face_img, debug=False):
        """
        Detecta a cor predominante na face do elo
        """
        if face_img is None:
            return 'indefinida', 0.0
            
        # Obter dimensões da imagem
        height, width = face_img.shape[:2]
        total_pixels = height * width
        
        # Pré-processar imagem
        hsv, blurred = self.preprocess_for_color(face_img)
        
        # Armazenar resultados de cada cor
        color_results = {}
        
        # Processar cores normais (amarelo, verde)
        for color_name, ranges in self.color_ranges.items():
            if color_name in ['vermelho1', 'vermelho2', 'branco']:
                continue
                
            # Criar máscara para a cor atual
            mask = cv2.inRange(hsv, ranges['lower'], ranges['upper'])
            
            # Operações morfológicas para limpar ruído
            kernel = np.ones((5,5), np.uint8)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
            
            # Calcular área da cor
            color_pixels = cv2.countNonZero(mask)
            color_percentage = color_pixels / total_pixels
            
            color_results[color_name] = color_percentage
            
            if debug:
                print(f"Porcentagem de {color_name}: {color_percentage:.2%}")
        
        # Processar vermelho (combinar os dois ranges)
        mask_red1 = cv2.inRange(hsv, self.color_ranges['vermelho1']['lower'],
                               self.color_ranges['vermelho1']['upper'])
        mask_red2 = cv2.inRange(hsv, self.color_ranges['vermelho2']['lower'],
                               self.color_ranges['vermelho2']['upper'])
        mask_red = cv2.bitwise_or(mask_red1, mask_red2)
        
        # Limpar máscara vermelha
        kernel = np.ones((5,5), np.uint8)
        mask_red = cv2.morphologyEx(mask_red, cv2.MORPH_OPEN, kernel)
        mask_red = cv2.morphologyEx(mask_red, cv2.MORPH_CLOSE, kernel)
        
        red_pixels = cv2.countNonZero(mask_red)
        red_percentage = red_pixels / total_pixels
        color_results['vermelho'] = red_percentage
        
        if debug:
            print(f"Porcentagem de vermelho: {red_percentage:.2%}")
        
        # Processar branco
        mask_white = cv2.inRange(hsv, self.color_ranges['branco']['lower'],
                                self.color_ranges['branco']['upper'])
        
        # Limpar máscara branca
        mask_white = cv2.morphologyEx(mask_white, cv2.MORPH_OPEN, kernel)
        mask_white = cv2.morphologyEx(mask_white, cv2.MORPH_CLOSE, kernel)
        
        white_pixels = cv2.countNonZero(mask_white)
        white_percentage = white_pixels / total_pixels
        color_results['branco'] = white_percentage
        
        if debug:
            print(f"Porcentagem de branco: {white_percentage:.2%}")
        
        # Encontrar cor predominante
        max_color = max(color_results.items(), key=lambda x: x[1])
        color_name, color_percentage = max_color
        
        # Verificar se a área é significativa
        if color_percentage < self.min_color_area:
            return 'indefinida', color_percentage
        
        if debug:
            print(f"\nCor detectada: {color_name} ({color_percentage:.2%})")
        
        return color_name, color_percentage

### Funções auxiliares e principais

#### 1. `load_images(data_dir='data')`
Carrega todas as imagens no formato `.png` de um diretório especificado. Retorna uma lista com as imagens carregadas e seus respectivos caminhos.

In [14]:
def load_images(data_dir='data'):
    """
    Carrega todas as imagens PNG do diretório especificado retornando os caminhos completos
    """
    images = []
    image_paths = []
    path = Path(data_dir)
    
    if not path.exists():
        print(f"Diretório {data_dir} não encontrado")
        return images, image_paths
    
    # Busca todos os arquivos .png no diretório e armazena seus caminhos completos
    for img_path in sorted(path.glob('*.png')):
        img = cv2.imread(str(img_path))
        if img is not None:
            images.append(img)
            image_paths.append(str(img_path))  # Convertendo Path para string do caminho completo
    
    if not images:
        print(f"Nenhuma imagem PNG encontrada em {data_dir}")
    else:
        print(f"Encontradas {len(images)} imagens em {data_dir}")
    
    return images, image_paths

#### 2. `find_elo_contour(image, debug=False)`
Localiza o contorno do Elo Maluco em uma imagem usando técnicas de processamento de bordas (Canny) e filtragem de contornos com base em área e proporção.

In [15]:
def find_elo_contour(image, debug=False):
    """
    Encontra o contorno do retângulo principal do Elo Maluco
    """
    try:
        # Converter para escala de cinza
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Aplicar blur suave para reduzir ruído mantendo bordas
        blurred = cv2.GaussianBlur(gray, (3, 3), 0)
        
        # Usar Canny para detecção de bordas
        edges = cv2.Canny(blurred, 30, 150)
        
        # Dilatar as bordas para conectar possíveis quebras
        kernel = np.ones((3,3), np.uint8)
        dilated = cv2.dilate(edges, kernel, iterations=1)
        
        # Encontrar contornos
        contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Filtrar contornos
        valid_contours = []
        for contour in contours:
            area = cv2.contourArea(contour)
            if area > 100:  # Reduzido o threshold de área mínima
                peri = cv2.arcLength(contour, True)
                approx = cv2.approxPolyDP(contour, 0.04 * peri, True)
                
                # Verificar se é aproximadamente retangular
                if len(approx) >= 4:
                    # Obter retângulo mínimo
                    rect = cv2.minAreaRect(contour)
                    box = cv2.boxPoints(rect)
                    box = np.int32(box)  # Alterado de np.int0 para np.int32
                    
                    # Verificar proporção do retângulo
                    width = rect[1][0]
                    height = rect[1][1]
                    if width == 0 or height == 0:
                        continue
                        
                    aspect_ratio = max(width, height) / min(width, height)
                    if 2 <= aspect_ratio <= 6:  # Proporção típica do Elo
                        valid_contours.append((area, box))
        
        if not valid_contours:
            print("Nenhum contorno válido encontrado")
            return None
        
        # Ordenar por área e pegar o maior
        valid_contours.sort(key=lambda x: x[0], reverse=True)
        elo_contour = valid_contours[0][1]
        
        if debug: #aqui
            debug_img = image.copy()
            cv2.drawContours(debug_img, [elo_contour], -1, (0, 255, 0), 2)
        
        return elo_contour
        
    except Exception as e:
        print(f"Erro ao encontrar contorno: {str(e)}")
        return None


#### 3. `extract_elo_region(image, contour, debug=False)`
Extrai a região retangular correspondente ao Elo Maluco com base no contorno encontrado. Aplica uma transformação de perspectiva para alinhar a região de forma vertical.


In [ ]:
def extract_elo_region(image, contour, debug=False):
    """
    Extrai a região do Elo Maluco usando o contorno encontrado com melhor precisão
    """
    try:
        if contour is None:
            print("Contorno não fornecido para extração")
            return None
            
        # Obter os pontos ordenados do retângulo
        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        box = np.int32(box)
        
        # Ordenar pontos em sentido horário começando pelo superior esquerdo
        center = np.mean(box, axis=0)
        
        # Ordenar pontos baseado no ângulo em relação ao centro
        def sort_points(points, center):
            return sorted(points, key=lambda p: np.arctan2(p[1] - center[1], p[0] - center[0]))
        
        box = np.array(sort_points(box, center))
        
        # Garantir que os pontos estão em ordem para formar um retângulo vertical
        # (superior-esquerdo, superior-direito, inferior-direito, inferior-esquerdo)
        if box[0][1] > box[1][1]:
            box = np.roll(box, 1, axis=0)
            
        if debug: #aqui
            debug_img = image.copy()
            for i, point in enumerate(box):
                cv2.circle(debug_img, tuple(map(int, point)), 5, (0, 255, 0), -1)
                cv2.putText(debug_img, str(i), tuple(map(int, point)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        
        # Calcular a largura e altura do retângulo
        width = int(max(
            np.linalg.norm(box[0] - box[1]),
            np.linalg.norm(box[2] - box[3])
        ))
        height = int(max(
            np.linalg.norm(box[1] - box[2]),
            np.linalg.norm(box[3] - box[0])
        ))
        
        # Definir pontos de destino para a transformação perspectiva
        dst_points = np.array([
            [0, 0],
            [width-1, 0],
            [width-1, height-1],
            [0, height-1]
        ], dtype=np.float32)
        
        # Calcular matriz de transformação perspectiva
        matrix = cv2.getPerspectiveTransform(box.astype(np.float32), dst_points)
        
        # Aplicar transformação perspectiva
        warped = cv2.warpPerspective(image, matrix, (width, height))
        
        # Garantir que a imagem está na orientação vertical correta
        if width > height:
            warped = cv2.rotate(warped, cv2.ROTATE_90_CLOCKWISE)
            width, height = height, width
            
        # Verificar se as proporções estão corretas
        aspect_ratio = height / width if width > 0 else 0
        if aspect_ratio < 2.5:  # O Elo deve ser mais alto que largo
            print(f"Proporções suspeitas (aspect_ratio = {aspect_ratio:.2f})")
            if debug:
                print("Tentando corrigir orientação...")
            warped = cv2.rotate(warped, cv2.ROTATE_90_CLOCKWISE)
        
        # Verificar se há faces suficientes
        min_face_height = 20  # altura mínima esperada para uma face
        if warped.shape[0] < 4 * min_face_height:
            print(f"Altura total ({warped.shape[0]}px) muito pequena para 4 faces")
            return None
            
        return warped
        
    except Exception as e:
        print(f"Erro ao extrair região: {str(e)}")
        if debug:
            print("Detalhes do erro:")
            import traceback
            traceback.print_exc()
        return None

#### 4. `split_faces(elo_region, debug=False)`
Divide a região do Elo Maluco em quatro partes iguais (faces) com base na altura total da região. Garante que as proporções de cada face sejam válidas.

In [17]:
def split_faces(elo_region, debug=False):
    """
    Divide a região do Elo em 4 faces com verificações melhoradas
    """
    try:
        if elo_region is None:
            print("Região do Elo não fornecida para divisão")
            return []
        
        height, width = elo_region.shape[:2]
        
        # Verificar orientação
        if width > height:
            print("Corrigindo orientação da imagem...")
            elo_region = cv2.rotate(elo_region, cv2.ROTATE_90_CLOCKWISE)
            height, width = width, height
        
        # Verificar dimensões mínimas
        if height < 80 or width < 20:  # 20 pixels mínimos por face em altura
            print(f"Dimensões muito pequenas: {width}x{height}")
            return []
        
        # Calcular altura de cada face
        face_height = height // 4
        
        # Adicionar uma pequena sobreposição para evitar cortes em bordas importantes
        overlap = 2
        faces = []
        
        for i in range(4):
            y_start = max(0, i * face_height - overlap)
            y_end = min(height, (i + 1) * face_height + overlap)
            
            face = elo_region[y_start:y_end, :]
            
            # Verificar se a face é válida
            if face.size == 0 or face.shape[0] < 10 or face.shape[1] < 10:
                print(f"Face {i+1} inválida: dimensões {face.shape}")
                continue
            
            # Remover a sobreposição nas faces extremas
            if i == 0:  # Primeira face
                face = face[overlap:, :]
            elif i == 3:  # Última face
                face = face[:-overlap, :]
            
            faces.append(face)
        
        # Verificação final
        if len(faces) != 4:
            print(f"Número incorreto de faces extraídas: {len(faces)}")
            return []
        
        return faces
        
    except Exception as e:
        print(f"Erro ao dividir faces: {str(e)}")
        return []

#### 5. `process_image(image, image_name="", debug=False)`
Processa uma única imagem:
- Localiza o contorno do Elo.
- Extrai a região do Elo.
- Divide a região em faces.
Retorna as faces extraídas para análise posterior.

In [18]:
def process_image(image, image_name="", debug=False):
    """
    Processa uma única imagem com melhor tratamento de erros
    """
    print(f"\nProcessando imagem: {image_name}")
    
    if image is None:
        print("Imagem inválida fornecida")
        return []
    
    # Encontrar o contorno do Elo
    print("Procurando contorno...")
    elo_contour = find_elo_contour(image, debug)
    
    if elo_contour is None:
        print("Não foi possível encontrar o contorno do Elo")
        return []
    
    # Extrair a região do Elo
    print("Extraindo região...")
    elo_region = extract_elo_region(image, elo_contour, debug)
    
    if elo_region is None:
        print("Não foi possível extrair a região do Elo")
        return []
    
    # Dividir em faces
    print("Dividindo faces...")
    faces = split_faces(elo_region, debug)
    
    if not faces:
        print("Não foi possível dividir as faces")
        return []
    
    cv2.destroyAllWindows()
    
    print(f"Processamento concluído: {len(faces)} faces encontradas")
    return faces

#### 6. `process_elo(image_path, pattern_matcher, color_detector, debug=False)`
Processa uma imagem completa do Elo Maluco:
- Carrega a imagem do caminho fornecido.
- Utiliza `PatternMatcher` para identificar padrões nas faces.
- Utiliza `EloColorDetector` para detectar as cores predominantes.
Retorna uma lista com os resultados para cada face.

In [19]:
def process_elo(image_path, pattern_matcher, color_detector, debug=False):
    """
    Processa uma imagem do elo detectando padrões e cores
    """
    # Carregar imagem
    image = cv2.imread(str(image_path))
    if image is None:
        print(f"Não foi possível carregar a imagem: {image_path}")
        return []
    
    # Processar imagem para encontrar faces
    faces = process_image(image, debug)
    
    if not faces:
        print("Não foram encontradas faces na imagem")
        return []
    
    # Analisar cada face
    results = []
    for i, face in enumerate(faces):
        print(f"\nAnalisando face {i+1}")
        
        # Detectar padrão
        pattern, similarity = pattern_matcher.match_pattern(face, debug)
        
        # Detectar cor
        color, color_percentage = color_detector.detect_color(face, debug)
        
        results.append({
            'posicao': i + 1,
            'padrao': pattern,
            'similaridade_padrao': similarity,
            'cor': color,
            'confianca_cor': color_percentage
        })
        
        print(f"Face {i+1}:")
        print(f"  Padrão: {pattern} (similaridade: {similarity:.3f})")
        print(f"  Cor: {color} (confiança: {color_percentage:.2%})")
    
    return results

#### 7. `generate_xml_state(results)`
Gera uma representação em formato XML do estado atual do Elo Maluco com base nos resultados de padrões e cores detectados.

In [20]:
def generate_xml_state(results):
    """
    Gera o XML do estado atual do Elo Maluco
    """
    # Criar o elemento raiz
    root = ET.Element("EloMaluco")
    estado_atual = ET.SubElement(root, "EstadoAtual")
    
    # Mapeamento de cor e posição para código
    color_map = {
        'vermelho': 'vm',
        'verde': 'vr',
        'amarelo': 'am',
        'branco': 'br'
    }
    
    position_map = {
        'superior': 's',
        'intermediaria': 'm',
        'inferior': 'i'
    }
    
    # Processar resultados em grupos de 4 faces
    current_row = None
    for i, result in enumerate(results):
        if i % 4 == 0:
            current_row = ET.SubElement(estado_atual, "row")
            
        # Criar elemento col
        col = ET.SubElement(current_row, "col")
        
        # Gerar código da face
        if result['padrao'] == 'vazia':
            col.text = 'vzo'
        else:
            # Combinar código de cor com código de posição
            color_code = color_map.get(result['cor'], 'v')  # Default para vermelho se cor indefinida
            position_code = position_map.get(result['padrao'], 'm')  # Default para intermediaria se indefinido
            col.text = color_code + position_code
    
    # Converter para string formatada
    xml_str = minidom.parseString(ET.tostring(root, encoding='UTF-8')).toprettyxml(indent="    ")
    return xml_str

#### 8. `save_xml_state(results, output_file='estado_atual.xml')`
Salva o XML gerado pela função `generate_xml_state` em um arquivo. O nome padrão do arquivo é `estado_atual.xml`.

In [21]:
def save_xml_state(results, output_file='estado_atual.xml'):
    """
    Salva o estado atual do Elo Maluco em um arquivo XML
    """
    xml_content = generate_xml_state(results)
    
    with open(output_file, 'w', encoding='UTF-8') as f:
        f.write(xml_content)
    
    print(f"\nEstado salvo em {output_file}")

#### 9. `main()`
Função principal que:
- Inicializa os detectores de padrões e cores.
- Carrega as imagens do diretório `data`.
- Processa cada imagem, gerando os resultados de padrões e cores.
- Salva o estado final no formato XML.

In [22]:
def main():
    # Inicializar detectores
    pattern_matcher = PatternMatcher()
    color_detector = EloColorDetector()
    
    # Carregar e processar todas as imagens do diretório data
    images, image_paths = load_images()
    
    if not images:
        print("Nenhuma imagem encontrada para processar")
        return
    
    # Lista para armazenar todos os resultados
    all_results = []
    
    # Processar cada imagem
    for image_path in image_paths:
        print(f"\nProcessando imagem: {Path(image_path).name}")
        results = process_elo(image_path, pattern_matcher, color_detector, debug=True)
        
        if results:
            print(f"\nResultados para {Path(image_path).name}:")
            for result in results:
                print(f"Face {result['posicao']}:")
                print(f"  Padrão: {result['padrao']} (similaridade: {result['similaridade_padrao']:.3f})")
                print(f"  Cor: {result['cor']} (confiança: {result['confianca_cor']:.2%})")
                all_results.append(result)
        else:
            print(f"Falha no processamento da imagem {Path(image_path).name}")
        
        print("-" * 50)
    
    # Gerar e salvar XML
    if all_results:
        save_xml_state(all_results)
    
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
    

Template 'superior' carregado com sucesso
Template 'intermediaria' carregado com sucesso
Template 'inferior' carregado com sucesso
Template 'vazia' carregado com sucesso
Encontradas 4 imagens em data

Processando imagem: Ex_input01_01.png

Processando imagem: True
Procurando contorno...
Extraindo região...
Dividindo faces...
Processamento concluído: 4 faces encontradas

Analisando face 1

Similaridades encontradas:
superior: 14.662
intermediaria: 9.138
inferior: 10.093
vazia: 0.458
Porcentagem de amarelo: 0.00%
Porcentagem de verde: 0.00%
Porcentagem de vermelho: 68.31%
Porcentagem de branco: 0.00%

Cor detectada: vermelho (68.31%)
Face 1:
  Padrão: superior (similaridade: 14.662)
  Cor: vermelho (confiança: 68.31%)

Analisando face 2

Similaridades encontradas:
superior: 14.316
intermediaria: 8.746
inferior: 10.094
vazia: 0.471
Porcentagem de amarelo: 0.00%
Porcentagem de verde: 67.34%
Porcentagem de vermelho: 0.00%
Porcentagem de branco: 7.42%

Cor detectada: verde (67.34%)
Face 2:
 